In [11]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from nltk.corpus import stopwords

In [12]:
df = pd.read_csv('datasheets/corpus_prepared.csv', sep='\t')
del df['Unnamed: 0']

In [13]:
texts = [list(map(lambda x: x.strip().strip('"').strip("'"), t.lstrip("['").rstrip("]").split(r"',"))) for t in df['text']]

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(list(texts))

# Create Corpus
#texts = df['text']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:10])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(1, 1), (6, 1), (17, 1), (22, 1)], [(4, 1), (8, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)], [(8, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(8, 1), (37, 2), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(37, 1), (38, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1)], [(28, 1), (37, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1)]]


[[('aim', 1),
  ('britain', 1),
  ('foremost', 1),
  ('learn', 1),
  ('make', 1),
  ('society', 1),
  ('world', 1)]]

In [20]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=5, 
                                            update_every=1, 
                                            chunksize=1, 
                                            passes=1)

In [21]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"service" + 0.032*"include" + 0.022*"social" + 0.021*"strategy" + '
  '0.020*"health" + 0.016*"area" + 0.016*"rural" + 0.015*"education" + '
  '0.014*"gaeltacht" + 0.014*"establish"'),
 (1,
  '0.040*"system" + 0.035*"tax" + 0.026*"state" + 0.025*"new" + 0.019*"future" '
  '+ 0.019*"need" + 0.017*"time" + 0.016*"create" + 0.016*"bank" + '
  '0.016*"development"'),
 (2,
  '0.066*"ireland" + 0.045*"people" + 0.042*"community" + 0.040*"ensure" + '
  '0.020*"make" + 0.016*"live" + 0.016*"must" + 0.013*"number" + 0.013*"end" + '
  '0.013*"school"'),
 (3,
  '0.030*"government" + 0.028*"right" + 0.027*"support" + 0.027*"equality" + '
  '0.019*"island" + 0.018*"introduce" + 0.018*"year" + 0.016*"base" + '
  '0.016*"body" + 0.015*"provide"'),
 (4,
  '0.061*"irish" + 0.035*"public" + 0.030*"increase" + 0.026*"economic" + '
  '0.021*"policy" + 0.017*"fully" + 0.016*"citizen" + 0.015*"sector" + '
  '0.015*"work" + 0.012*"believe"')]


In [22]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.016650 -0.141582       1        1  20.423105
1     -0.283171  0.226971       2        1  20.182117
3      0.037552  0.003369       3        1  19.915672
4     -0.064343 -0.258730       4        1  19.765026
0      0.293312  0.169972       5        1  19.714069, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
1378  Default  3745.000000         ireland  3745.000000  30.0000  30.0000
1382  Default  3394.000000           irish  3394.000000  29.0000  29.0000
321   Default  2590.000000          people  2590.000000  28.0000  28.0000
221   Default  2494.000000         service  2494.000000  27.0000  27.0000
212   Default  2403.000000       community  2403.000000  26.0000  26.0000
141   Default  2276.000000          system  2276.000000  25.0000  25.0000
94    Default  2280.000000          ensure  2280.000000  24.0000  24.0000
106   Default  1942.000000          public  1942.000000  23.0000  23.0000
36    Default  1963.000000             tax  1963.000000  22.0000  22.0000
301   Default  1790.000000         include  1790.000000  21.0000  21.0000
24    Default  1665.000000      government  1665.000000  20.0000  20.0000
53    Default  1636.000000        increase  1636.000000  19.0000  19.0000
521   Default  1565.000000           right  1565.000000  18.0000  18.0000
149   Default  1510.000000         support  1510.000000  17.0000  17.0000
402   Default  1458.000000        economic  1458.000000  16.0000  16.0000
293   Default  1448.000000           state  1448.000000  15.0000  15.0000
254   Default  1394.000000             new  1394.000000  14.0000  14.0000
285   Default  1637.000000        equality  1637.000000  13.0000  13.0000
579   Default  1226.000000          social  1226.000000  12.0000  12.0000
9     Default  1171.000000          policy  1171.000000  11.0000  11.0000
419   Default  1139.000000        strategy  1139.000000  10.0000  10.0000
715   Default  1123.000000          health  1123.000000   9.0000   9.0000
4     Default  1116.000000            make  1116.000000   8.0000   8.0000
2209  Default  1049.000000          island  1049.000000   7.0000   7.0000
171   Default  1057.000000          future  1057.000000   6.0000   6.0000
57    Default  1054.000000            need  1054.000000   5.0000   5.0000
358   Default  1006.000000       introduce  1006.000000   4.0000   4.0000
37    Default   983.000000            year   983.000000   3.0000   3.0000
208   Default   955.000000           fully   955.000000   2.0000   2.0000
324   Default   952.000000            time   952.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
715    Topic5  1123.257568          health  1123.991211   1.6232  -3.8937
419    Topic5  1138.817993        strategy  1139.566650   1.6232  -3.8800
680    Topic5   860.703125            area   861.459534   1.6230  -4.1600
8      Topic5   834.783203       education   835.540588   1.6229  -4.1906
187    Topic5   760.115295       establish   760.861145   1.6229  -4.2843
595    Topic5   751.974243             law   752.750366   1.6228  -4.2950
757    Topic5   855.478210           rural   856.365234   1.6228  -4.1661
548    Topic5   678.041199          centre   678.779175   1.6227  -4.3985
38     Topic5   666.163147           child   666.908081   1.6227  -4.4162
143    Topic5   584.482666          access   585.228577   1.6226  -4.5470
639    Topic5   568.020020      protection   568.767517   1.6225  -4.5756
153    Topic5   555.860901         require   556.615295   1.6225  -4.5972
77     Topic5   535.633057         economy   536.370667   1.6225  -4.6343
3309   Topic5   704.165894          county   705.195496   1.6224  -4.3607
488    Topic5   506.897034  infrastructure   507.658081   1.6223  -4.6894
105    Topic5   491.506348 